In [20]:
import requests
import json
import time
import copy
from bs4 import BeautifulSoup as bs

In [30]:
SIDO_DICTONARY = {'서울':'11', '인천':'28', '세종':'36', '대전':'30', '광주':'29', '울산':'31', 
                  '대구':'27', '부산':'26', '경기':'41', '강원':'42', '충북':'43', '충남':'44', 
                  '충북':'43', '경북':'47', '전북':'45', '전남':'46', '경남':'48'}
# 시에서 구을 검색할때 코드
#BJD_CODE = 'SGG'
#구에서 동을 검색할때 코드
#BJD_CODE = 'EMD'

In [31]:
urlSido = 'http://www.k-apt.go.kr/cmmn/bjd/getBjdList.do?' # 동까지 조회 할때 사용하는 url
urlApt = 'http://www.k-apt.go.kr/kaptinfo/getKaptList.do?'  # 아파트리스트 가져올때 사용하는 url

# urlMonth = 'http://www.k-apt.go.kr/apiinfo/openCostMonthUseAve.do?' # 월별 에너지량
# url = 'http://www.k-apt.go.kr/apiinfo/openCostEnergyUseStatistics.do?' # 에너지사용 통계
# url = 'http://www.k-apt.go.kr/apiinfo/openCostYearUseAve.do' # 연도 에너리량

In [ ]:
# 페이지로 응답오는 화면인경우 리다이렉트로 실해앟게 되어 있음..
# data = {'go_url':'/kaptinfo/openkaptinfo.do','bjd_code': '1168010300', 'kapt_code': 'A13593908', 'search_date': '202203'}  # 서울시 강남구 개포동
# response = web_request(method_name='POST', url='http://www.k-apt.go.kr/cmmn/selectKapt.do?', dict_data=data)
#http://www.k-apt.go.kr/cmmn/selectKapt.do
#http://www.k-apt.go.kr/kaptinfo/openkaptinfo.do
# '/kaptinfo/openkaptinfo.do? #우리단지 기본정보
# '/kaptinfo/openkaptinfoBid.do?'# 우리단지 입찰공고
# '/kaptinfo/openKaptMng.do?' #관리시설정보
# '/kaptinfo/openKaptSetday.do?' #주택인도일정보
# '/kaptinfo/openKaptGrEng.do?' #녹색/에너지정보
# '/kaptinfo/openKaptMngRule.do?' #관리규약
# '/kaptinfo/openKaptRep.do?' #입주자대표회의 구성인원

#<form id="cmnmMapFrm" method="post" action="/cmmn/selectKapt.do">
	#<input type="hidden" value="" name="go_url" id="go_url"> 
	#<input type="hidden" value="" name="bjd_code" id="bjd_code"> 
	#<input type="hidden" value="" name="kapt_code" id="kapt_code">
	#<input type="hidden" value="" name="search_date" id="search_date">
#</form>


In [56]:
def web_request(method_name, url, dict_data, is_urlencoded=True):
    """Web GET or POST request를 호출 후 그 결과를 dict형으로 반환 """
    method_name = method_name.upper()  # 메소드이름을 대문자로 바꾼다
    if method_name not in ('GET', 'POST'):
        raise Exception('method_name is GET or POST plz...')

    if method_name == 'GET':  # GET방식인 경우
        response = requests.get(url=url, params=dict_data)
    elif method_name == 'POST':  # POST방식인 경우
        if is_urlencoded is True:
            response = requests.post(url=url, data=dict_data,
                                     headers={'Content-Type': 'application/x-www-form-urlencoded'})
        else:
            response = requests.post(url=url, data=json.dumps(dict_data), headers={'Content-Type': 'application/json'})

    dict_meta = {'status_code': response.status_code, 'ok': response.ok, 'encoding': response.encoding,
                 'Content-Type': response.headers['Content-Type']}
    if 'json' in str(response.headers['Content-Type']):  # JSON 형태인 경우
        return {**dict_meta, **response.json()}
    else:  # 문자열 형태인 경우
        return {**dict_meta, **{'text': response.text}}

In [33]:
def getSidoList(paramCode, paramGbn):
    data = {'bjd_code': paramCode, 'bjd_gbn': paramGbn}  # 서울시
    # data = {'bjd_code': '11', 'bjd_gbn':'SGG'}           # 서울시
    # data = {'bjd_code': '11680', 'bjd_gbn':'EMD'}         # 서울시 강남구

    response = web_request(method_name='POST', url=urlSido, dict_data=data)
    if response['ok'] == True:
        # 성공 응답 시 액션
        result = json.loads(response['text'])
        return result
    else:
        # 실패 응답 시 액션
        pass

In [ ]:
def getAptList(paramCode, srchDate):
    # data = {'bjd_code': '1168010300'}         # 서울시 강남구 개포동
    data = {'bjd_code': paramCode, 'search_date': srchDate}  # 서울시 강남구 개포동
    response = web_request(method_name='POST', url=urlApt, dict_data=data)
    if response['ok'] == True:
        # 성공 응답 시 액션
        result = json.loads(response['text'])
        return result
    else:
        pass

In [36]:
# 1. 시에서 전체구를 조회하는 리스트를 만든다.
# 시에서 구를 검색하니 SGG값을 이용
SIDO_KEY = SIDO_DICTONARY['서울']
SIDO_CODE = 'SGG'

print('Start guList')
GU_LIST = getSidoList(SIDO_KEY, SIDO_CODE)
print('Start guList')

{'resultList': [{'CODE': '680', 'CODE_VALUE': '강남구'}, {'CODE': '740', 'CODE_VALUE': '강동구'}, {'CODE': '305', 'CODE_VALUE': '강북구'}, {'CODE': '500', 'CODE_VALUE': '강서구'}, {'CODE': '620', 'CODE_VALUE': '관악구'}, {'CODE': '215', 'CODE_VALUE': '광진구'}, {'CODE': '530', 'CODE_VALUE': '구로구'}, {'CODE': '545', 'CODE_VALUE': '금천구'}, {'CODE': '350', 'CODE_VALUE': '노원구'}, {'CODE': '320', 'CODE_VALUE': '도봉구'}, {'CODE': '230', 'CODE_VALUE': '동대문구'}, {'CODE': '590', 'CODE_VALUE': '동작구'}, {'CODE': '440', 'CODE_VALUE': '마포구'}, {'CODE': '410', 'CODE_VALUE': '서대문구'}, {'CODE': '650', 'CODE_VALUE': '서초구'}, {'CODE': '200', 'CODE_VALUE': '성동구'}, {'CODE': '290', 'CODE_VALUE': '성북구'}, {'CODE': '710', 'CODE_VALUE': '송파구'}, {'CODE': '470', 'CODE_VALUE': '양천구'}, {'CODE': '560', 'CODE_VALUE': '영등포구'}, {'CODE': '170', 'CODE_VALUE': '용산구'}, {'CODE': '380', 'CODE_VALUE': '은평구'}, {'CODE': '110', 'CODE_VALUE': '종로구'}, {'CODE': '140', 'CODE_VALUE': '중구'}, {'CODE': '260', 'CODE_VALUE': '중랑구'}]}


In [37]:
#2. 각구별로 돌아가면서 동을 출력하는 리스트를 만든다.
# 구에서 동을 검색하니 EMD값을 이용
DONG_LIST = []
DONG_CODE = 'EMD'

print('Start dongList')
for gu in GU_LIST['resultList']:
    time.sleep(5)
    gu['CODE'] = SIDO_KEY + gu['CODE']
    dongList = getSidoList(gu['CODE'], DONG_CODE)
    for dong in dongList['resultList']:
        dong['CODE'] = gu['CODE'] + dong['CODE']
        DONG_LIST.append(dong)
print('End dongList')

11680
{'resultList': [{'CODE': '103', 'CODE_VALUE': '개포동'}, {'CODE': '108', 'CODE_VALUE': '논현동'}, {'CODE': '106', 'CODE_VALUE': '대치동'}, {'CODE': '118', 'CODE_VALUE': '도곡동'}, {'CODE': '105', 'CODE_VALUE': '삼성동'}, {'CODE': '111', 'CODE_VALUE': '세곡동'}, {'CODE': '115', 'CODE_VALUE': '수서동'}, {'CODE': '107', 'CODE_VALUE': '신사동'}, {'CODE': '110', 'CODE_VALUE': '압구정동'}, {'CODE': '101', 'CODE_VALUE': '역삼동'}, {'CODE': '113', 'CODE_VALUE': '율현동'}, {'CODE': '114', 'CODE_VALUE': '일원동'}, {'CODE': '112', 'CODE_VALUE': '자곡동'}, {'CODE': '104', 'CODE_VALUE': '청담동'}]}
{'resultList': [{'CODE': '103', 'CODE_VALUE': '개포동'}, {'CODE': '108', 'CODE_VALUE': '논현동'}, {'CODE': '106', 'CODE_VALUE': '대치동'}, {'CODE': '118', 'CODE_VALUE': '도곡동'}, {'CODE': '105', 'CODE_VALUE': '삼성동'}, {'CODE': '111', 'CODE_VALUE': '세곡동'}, {'CODE': '115', 'CODE_VALUE': '수서동'}, {'CODE': '107', 'CODE_VALUE': '신사동'}, {'CODE': '110', 'CODE_VALUE': '압구정동'}, {'CODE': '101', 'CODE_VALUE': '역삼동'}, {'CODE': '113', 'CODE_VALUE': '율현동'}, {'CODE': 

In [28]:
#3. 각동별로 아파트 정보를 년월기준으로 조회한다.
# 기준월을 세팅한다.
APT_LIST = []
SEARCH_DATE = '202203'

print('Start aptList')
for dong in DONG_LIST:
    time.sleep(5)
    temp = getAptList(dong['CODE'], SEARCH_DATE)
    for aptInfo in temp['resultList']:
        APT_LIST.append(aptInfo)
print('End aptList')

[]


In [ ]:
print('Start get APT engeryInfo')
for aptInfo in APT_LIST:
    break
    data = {'viewType': '1', 'bjdCode': aptInfo['BJD_CODE'], 'kaptCode': aptInfo['KAPT_CODE'], 'searchDate': SEARCH_DATE}  # 서울시 강남구 개포동
    # data = {'viewType': '1', 'search_date': searchDate, 'bjdCode': '', 'kaptCode': ''}  # 서울시 강남구 개포동
    response = requests.get(url=urlMonth, params=data)
    #time.sleep(5)
    #bjdList = aptInfo['BJD_NAME'].split(' ')
    #createResultByExcel.inputDataInExcel(bjdList[0],bjdList[1],bjdList[2],aptInfo['KAPT_NAME'], inqResult)
print('End get APT engeryInfo')

In [57]:
# 우리단지 기본정보
data = {'go_url':'/kaptinfo/openkaptinfo.do','bjd_code': '1168010300', 'kapt_code': 'A13593908', 'search_date': '202203'}  # 서울시 강남구 개포동
response = web_request(method_name='POST', url='http://www.k-apt.go.kr/cmmn/selectKapt.do?', dict_data=data2)

In [59]:
temp = copy.deepcopy(response)
soup = bs(temp['text'], "html.parser")

In [ ]:
# 관리시설정보
data = {'go_url':'/kaptinfo/openKaptMng.do','bjd_code': '1168010300', 'kapt_code': 'A13593908', 'search_date': '202203'}  # 서울시 강남구 개포동
response2 = web_request(method_name='POST', url='http://www.k-apt.go.kr/cmmn/selectKapt.do?', dict_data=data)
temp2 = copy.deepcopy(response2)
soup2 = bs(temp2['text'], "html.parser")

In [175]:
elements = soup.find("div", "subCbox")
td_values = elements.find_all('td')
tr_values = elements.find_all('tr')

print(elements.find_all('script'))
#for idx, item in enumerate(elements):            # enumerate를 사용하면 해당 값의 인덱스를 알 수 있다.
#    ths = item.find_all('th')
#    tds = item.find_all('td')
#    print(tds)

#td_values[0].text.strip() #명칭
## 법정동주소
#td_values[4].text.strip() #분양형태
#td_values[5].text.strip() #관리방식
#td_values[6].text.strip() #난방방식
#td_values[7].text.strip() #복도유형
#td_values[8].text.strip() #연면적
#td_values[10].text.strip().split('\n')[0] #동수/세대수
#td_values[10].text.strip().split('\n')[2] #주거전용면적
#td_values[12].text.strip() #시공사/시행사
#td_values[16].text.strip().split('\n')[0] #면적별 세대현황(전용면적 기준)
#td_values[16].text.strip().split('\n')[1] #면적별 세대현황(전용면적 기준)
#td_values[16].text.strip().split('\n')[2] #면적별 세대현황(전용면적 기준)
#td_values[16].text.strip().split('\n')[3] #면적별 세대현황(전용면적 기준)

[<script>
                            var dataB="", dataR="";
                            var arrGbn = new Array();
                            var arrAddr = new Array();
//                             for (var i=0 ; i < 10 ; i++) {
                           	if ("B" != "") {
                           		arrGbn.push("B");
                           		arrAddr.push("서울특별시 강남구 개포동 12 대치아파트");
                           	}
                           	if ("R" != "") {
                           		arrGbn.push("R");
                           		arrAddr.push("서울특별시 강남구 개포로109길 5");
                           	}
                           	if ("" != "") {
                           		arrGbn.push("");
                           		arrAddr.push("");
                           	}
                           	if ("" != "") {
                           		arrGbn.push("");
                           		arrAddr.push("");
                           	}
                           	if ("" != "") {
          

In [91]:
#soup2
#kaptMngTbl

elements2 = soup2.find("div", "subCbox")
print(elements2)


<div class="subCbox">
<!--서브컨텐츠시작-->
<h4 class="tle">관리사항</h4>
<table class="contTbl" id="kaptMngTbl">
<caption>관리시설정보</caption>
<colgroup>
<col class="wp20"/>
<col class="wp30"/>
<col class="wp20"/>
<col class="wp30"/>
</colgroup>
<tbody>
<tr>
<th scope="row">일반관리</th>
<td colspan="3">
<ul class="ul01 ul01_fl">
<li><b>관리방식 : </b><span id="code_mgr"></span></li>
<li><b>인원 : </b><span id="kapt_mgr_cnt"></span></li>
<li><b>계약업체 : </b><span id="kapt_ccompany"></span></li>
</ul>
</td>
</tr>
<tr>
<th scope="row">경비관리</th>
<td colspan="3">
<ul class="ul01 ul01_fl">
<li><b>관리방식 : </b><span id="code_sec"></span></li>
<li><b>인원 : </b><span id="kaptd_scnt"></span></li>
<li><b>계약업체 : </b><span id="kaptd_sec_com"></span></li>
</ul>
</td>
</tr>
<tr>
<th scope="row">청소관리</th>
<td colspan="3">
<ul class="ul01 ul01_fl">
<li><b>관리방식 : </b><span id="code_clean"></span></li>
<li><b>인원 : </b><span id="kaptd_clcnt"></span></li>
<li><b>음식물처리방법 : </b><span id="code_garbage"></span></li>
</ul>
</td>
</tr>
<tr